# Study Catalan Elections Participation Dataset

Load libraries:

In [1]:
import pandas as pd
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import logging
from unidecode import unidecode

pp = pprint.PrettyPrinter(indent=2)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

Load the dataset:

In [2]:
df = pd.read_pickle('../../data/raw/catalan-elections-participation.pkl')
df_original = df.copy()

## Dataset Structure

Visualize the structure of the dataset:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930819 entries, 0 to 930818
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   index_autonumeric       930819 non-null  Int64 
 1   id_eleccio              930819 non-null  string
 2   nom_eleccio             930819 non-null  string
 3   id_nivell_territorial   930819 non-null  string
 4   nom_nivell_territorial  930819 non-null  string
 5   territori_codi          930819 non-null  string
 6   territori_nom           930819 non-null  string
 7   districte               855641 non-null  Int64 
 8   seccio                  868235 non-null  Int64 
 9   cens_electoral          930819 non-null  Int64 
 10  votants                 930819 non-null  Int64 
 11  abstencio               930819 non-null  Int64 
 12  vots_nuls               930819 non-null  Int64 
 13  vots_blancs             930819 non-null  Int64 
 14  vots_candidatures       878103 non-n

| Column name        | Description                                 | Type      |
|--------------------|---------------------------------------------|-----------|
| TERRITORI_NOM      | Nom del territori                           | Text      |
| DISTRICTE          | Districte electoral                         | Text      |
| SECCIO             | Secció electoral                            | Text      |
| MESA               | Mesa electoral                              | Text      |
| CENS_ELECTORAL     | Cens del territori                          | Nombre    |
| PADRO              | Padró d'habitants del territori             | Nombre    |
| ESCONS             | Escons a escollir a la circumscripció       | Nombre    |
| NOMBRE_MESES       | Nombre de meses electorals                  | Nombre    |
| VOTANTS            | Nombre de votants                           | Nombre    |
| ABSTENCIO          | Nombre de persones que s'han abstingut      | Nombre    |
| VOTS_NULS          | Vots nuls                                   | Nombre    |
| VOTS_BLANCS        | Vots en blanc                               | Nombre    |
| VOTS_CANDIDATURES  | Vots a candidatures                         | Nombre    |
| VOTS_VALIDS        | Vots vàlids (a candidatures + en blanc)     | Nombre    |
| VOTS_PRIMER_AVAN   | Votants al primer avanç de participació     | Nombre    |
| HORA_PRIMER_AVAN   | Hora del primer avanç de participació       | Text      |
| VOTS_SEGON_AVAN    | Votants al segon avanç de participació      | Nombre    |
| HORA_SEGON_AVAN    | Hora del segon avanç de participació        | Text      |

## Types of Elections

First of all, we will divide `id_eleccio` into `type`, `year` and `sequential` as we did in the previous notebook. This will allow us to analyze the dataset by type of election.

In [4]:
df['type'] = df['id_eleccio'].str[:1]
df['year'] = df['id_eleccio'].str[1:5].astype(int)
df['sequential'] = df['id_eleccio'].str[5:]

Show the types of elections:

In [5]:
types = df[['type', 'nom_eleccio']].groupby(['type']).first()
print(types)
print(len(types))

                                   nom_eleccio
type                                          
A     Eleccions al Parlament de Catalunya 1995
C                         Eleccions Municipals
D          Eleccions a Diputacions Provincials
E               Eleccions al Parlament Europeu
G                      Eleccions Generals 2008
M                    Eleccions Municipals 2007
R               Referèndum Constitucional 1978
S                      Eleccions al Senat 1979
V     Eleccions al Consell General d'Aran 2023
9


Now we know that the dataset contains data from 9 different types of elections:

| Type | Election Type Name                          |
|------|---------------------------------------------|
| A    | Elections to the Parliament of Catalonia    |
| C    | Elections to the County Councils            |
| D    | Elections to the Provincial Councils        |
| E    | Elections to the European Parliament        |
| G    | Elections to the Congress                   |
| M    | Municipal Elections                         |
| R    | Constitutional Referendum                   |
| S    | Elections to the Senate                     |
| V    | Elections to the General Council of Aran    |

We can see that there is one type of election that is not present in the previous dataset, which is `R`. This type of election is called Constitutional Referendum (`Referèndum Constitucional 1978`).

## Filter by Election Type

As we explained on the previous notebook, we are only interested on the data from the elections to the Parliament of Catalonia ('A'), municipal elections ('M'), elections to the European Parliament ('E') and elections to the Congress ('G'), so we filter the dataset:

In [6]:
df = df[df['type'].isin(['M', 'E', 'A', 'G'])]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 678882 entries, 548 to 930234
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   index_autonumeric       678882 non-null  Int64 
 1   id_eleccio              678882 non-null  string
 2   nom_eleccio             678882 non-null  string
 3   id_nivell_territorial   678882 non-null  string
 4   nom_nivell_territorial  678882 non-null  string
 5   territori_codi          678882 non-null  string
 6   territori_nom           678882 non-null  string
 7   districte               624021 non-null  Int64 
 8   seccio                  633605 non-null  Int64 
 9   cens_electoral          678882 non-null  Int64 
 10  votants                 678882 non-null  Int64 
 11  abstencio               678882 non-null  Int64 
 12  vots_nuls               678882 non-null  Int64 
 13  vots_blancs             678882 non-null  Int64 
 14  vots_candidatures       678882 non-null

## Check for missing values

Now, we want to count the number of missing values in each column:

In [7]:
df.isnull().mean().sort_values(ascending=False) * 100

nombre_meses              93.256118
escons                    93.256118
padro                     93.115593
mesa                      49.661502
vots_segon_avan           22.344678
vots_primer_avan          22.344678
districte                  8.081080
seccio                     6.669348
index_autonumeric          0.000000
vots_valids                0.000000
year                       0.000000
type                       0.000000
hora_segon_avan            0.000000
hora_primer_avan           0.000000
vots_blancs                0.000000
vots_candidatures          0.000000
id_eleccio                 0.000000
vots_nuls                  0.000000
abstencio                  0.000000
votants                    0.000000
cens_electoral             0.000000
territori_nom              0.000000
territori_codi             0.000000
nom_nivell_territorial     0.000000
id_nivell_territorial      0.000000
nom_eleccio                0.000000
sequential                 0.000000
dtype: float64

Based on the previous table, we can see that `escons`, `nombre_meses` and `padro` have lots of empty values.

Also, we can see that `mesa` and `vots_segon_avan` and `vots_primer_avan` have a significant number of empty values.

We will study them to see if we can fill them with some value or if we need to remove them.

### Missing values in `escons`

We calculate the percentage of missing ``escons`` values by ``id_nivell_territorial``. The `NA` values represent 0.0% of the total:

In [9]:
df[["id_nivell_territorial", "nom_nivell_territorial"]].drop_duplicates()

,id_nivell_territorial,nom_nivell_territorial
548,DM,Districte Municipal
588,ME,Mesa
7876,SE,Secció
8459,CA,Catalunya
14911,PR,Província
14915,VE,Vegueria
14925,CO,Comarca
22989,MU,Municipi
112991,PR,Provincia
120016,ME,Municipi


In [37]:
(
    100
    * df[df["escons"].isnull()].value_counts("id_nivell_territorial")
    / df.value_counts("id_nivell_territorial")
).sort_values(ascending=False)

id_nivell_territorial
CL    100.000000
DM    100.000000
ME    100.000000
SE    100.000000
VE     23.232323
MU      2.172104
CA      2.083333
PR      2.083333
CO      0.152439
Name: count, dtype: float64

We can see that for ``Mesa``, ``Col·legi``, ``Districte Municipal`` and ``Secció`` the percentage of missing values is 100.0%. On the other territorial levels, the percentage of missing values is near 0.0%.

We can also study the percentage of missing values by election type (`type`):

In [11]:
100 * df[df["escons"].isnull()].value_counts("type") / df.value_counts("type")

type
G    93.117926
A    93.382706
M    92.577261
E    94.132990
Name: count, dtype: float64

We will filter out the territorial levels with 100.0% of missing values:

In [41]:
df_filtered = df[~df["id_nivell_territorial"].isin(["CL", "DM", "ME", "SE"])]
df_filtered.isnull().mean().sort_values(ascending=False) * 100

mesa                      99.967935
districte                 99.950834
seccio                    34.292433
vots_segon_avan           25.517315
vots_primer_avan          25.517315
nombre_meses               2.131253
escons                     2.131253
padro                      0.091920
index_autonumeric          0.000000
vots_valids                0.000000
year                       0.000000
type                       0.000000
hora_segon_avan            0.000000
hora_primer_avan           0.000000
vots_blancs                0.000000
vots_candidatures          0.000000
id_eleccio                 0.000000
vots_nuls                  0.000000
abstencio                  0.000000
votants                    0.000000
cens_electoral             0.000000
territori_nom              0.000000
territori_codi             0.000000
nom_nivell_territorial     0.000000
id_nivell_territorial      0.000000
nom_eleccio                0.000000
sequential                 0.000000
dtype: float64

After filtering, we can see that the percentage of missing values has been drastically reduced on almost every column. The only exceptions are `mesa` and `districte`, which are almost empty. That makes sense because we have removed th rows with electoral tables and districts information.

In [25]:
print(
    (
        100
        * df_filtered[df_filtered["escons"].isnull()].value_counts(
            "id_nivell_territorial"
        )
        / df_filtered["id_nivell_territorial"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df_filtered[df_filtered["escons"].isnull()].value_counts("type")
        / df_filtered["type"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df_filtered[df_filtered["escons"].isnull()].value_counts("territori_nom")
        / df_filtered["territori_nom"].value_counts()
    )
    .sort_values(ascending=False)
    .dropna()
)

id_nivell_territorial
VE    23.232323
MU     2.172104
CA     2.083333
PR     2.083333
CO     0.152439
Name: count, dtype: Float64
type
E    11.988964
G     0.127218
A     0.115723
M     0.064167
Name: count, dtype: Float64
territori_nom
CERA Tarragona      100.0
CERA Lleida         100.0
CERA Girona         100.0
CERA Catalunya      100.0
CERA Barcelona      100.0
                   ...   
Gironella         2.12766
Gisclareny        2.12766
Godall            2.12766
Golmés            2.12766
Tarragona         1.06383
Name: count, Length: 956, dtype: Float64


Most of the missing values are from ``CERA *`` and ``Residents Absents`` on the ``territori_nom`` column. This is logic because these "territories" are not physical places, but groups of people and therefore they don't have a number of seats to be elected.

In [13]:
territori_nom_escons_null = (
    (
        100
        * df_filtered[df_filtered["escons"].isnull()].value_counts("territori_nom")
        / df_filtered["territori_nom"].value_counts()
    )
    .sort_values(ascending=False)
    .dropna()
)
eleccions_escons_null = (
    (
        100
        * df_filtered[df_filtered["escons"].isnull()].value_counts("type")
        / df_filtered["type"].value_counts()
    )
    .sort_values(ascending=False)
    .dropna()
)
generals_escons_null = (
    df_filtered[(df_filtered["escons"].isnull()) & (df_filtered["type"] == "G")]
)
autonomiques_escons_null = (
    df_filtered[(df_filtered["escons"].isnull()) & (df_filtered["type"] == "A")]
)
europees_escons_null = (
    df_filtered[(df_filtered["escons"].isnull()) & (df_filtered["type"] == "E")]
)
print(
    (
        100
        * europees_escons_null[europees_escons_null["escons"].isnull()].value_counts("territori_nom")
        / europees_escons_null["territori_nom"].value_counts()
    )
    .sort_values(ascending=False)
)

territori_nom
Abella de la Conca     100.0
Rubí                   100.0
Riudoms                100.0
Riumors                100.0
Roca del Vallès, la    100.0
                       ...  
Garriguella            100.0
Garrigàs               100.0
Gavet de la Conca      100.0
Gavà                   100.0
Òrrius                 100.0
Name: count, Length: 947, dtype: Float64


Vegueries doesn't have seats to be elected, so on the catalan elections they have missing values for seats.

On the european elections, any municipality has seats to be elected, so they have missing values for seats.

In [24]:
df_2009 = df_filtered[df_filtered["year"] == 2009]

### Missing values in `nombre_meses`

Now we analize the missing values in `nombre_meses`. We calculate the percentage of missing ``nombre_meses`` values by ``id_nivell_territorial``. The `NA` values represent 0.0% of the total:

In [46]:
missing_column = "nombre_meses"
(
    100
    * df[df[missing_column].isnull()].value_counts(
        "id_nivell_territorial"
    )
    / df["id_nivell_territorial"].value_counts()
).sort_values(ascending=False)

id_nivell_territorial
CL        100.0
DM        100.0
ME        100.0
SE        100.0
VE    23.232323
MU     2.172104
CA     2.083333
PR     2.083333
CO     0.152439
Name: count, dtype: Float64

We can see that the percentage of missing values on territorial levels `CL`, `DM`, `ME` and `SE` is the same as in `escons`. We can proceed using the filtered dataset.

In [26]:
print(
    (
        100
        * df_filtered[df_filtered[missing_column].isnull()].value_counts(
            "id_nivell_territorial"
        )
        / df_filtered["id_nivell_territorial"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df_filtered[df_filtered[missing_column].isnull()].value_counts("type")
        / df_filtered["type"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df_filtered[df_filtered[missing_column].isnull()].value_counts("territori_nom")
        / df_filtered["territori_nom"].value_counts()
    )
    .sort_values(ascending=False)
    .dropna()
)

id_nivell_territorial
VE    23.232323
MU     2.172104
CA     2.083333
PR     2.083333
CO     0.152439
Name: count, dtype: Float64
type
E    11.988964
G     0.127218
A     0.115723
M     0.064167
Name: count, dtype: Float64
territori_nom
CERA Tarragona      100.0
CERA Lleida         100.0
CERA Girona         100.0
CERA Catalunya      100.0
CERA Barcelona      100.0
                   ...   
Gironella         2.12766
Gisclareny        2.12766
Godall            2.12766
Golmés            2.12766
Tarragona         1.06383
Name: count, Length: 956, dtype: Float64


It seems that the missing values are related to the same characteristics as the missing values in `escons`.

### Missing values in `padro`

We will study the missing values in `padro`. We start by seeing if the same territorial levels as `escons` have the same percentage of missing values:

In [45]:
missing_column = "padro"
(
    100
    * df[df[missing_column].isnull()].value_counts(
        "id_nivell_territorial"
    )
    / df["id_nivell_territorial"].value_counts()
).sort_values(ascending=False)

id_nivell_territorial
CL        100.0
DM        100.0
ME        100.0
SE        100.0
VE    23.232323
CA     2.083333
PR     2.083333
CO     0.152439
MU     0.026983
Name: count, dtype: Float64

We can see that the percentage of missing values on territorial levels `CL`, `DM`, `ME` and `SE` is the same as in `escons`. We can proceed using the filtered dataset.

In [39]:
print(
    (
        100
        * df_filtered[df_filtered[missing_column].isnull()].value_counts(
            "id_nivell_territorial"
        )
        / df_filtered["id_nivell_territorial"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df_filtered[df_filtered[missing_column].isnull()].value_counts("type")
        / df_filtered["type"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df_filtered[df_filtered[missing_column].isnull()].value_counts("territori_nom")
        / df_filtered["territori_nom"].value_counts()
    )
    .sort_values(ascending=False)
    .dropna()
)

id_nivell_territorial
VE    23.232323
CA     2.083333
PR     2.083333
CO     0.152439
MU     0.026983
Name: count, dtype: Float64
type
A    0.115723
G    0.100435
E    0.075245
M    0.064167
Name: count, dtype: Float64
territori_nom
CERA Barcelona                        100.0
CERA Catalunya                        100.0
CERA Girona                           100.0
CERA Lleida                           100.0
CERA Tarragona                        100.0
Alt Pirineu i territori d'Aran    16.666667
Camp de Tarragona                 16.666667
Catalunya Central                 16.666667
Terres de l'Ebre                  16.666667
Residents Absents                 11.949686
Barcelona                          1.886792
Girona                             1.886792
Lleida                             1.886792
Name: count, dtype: Float64


The column ``padro`` doesn't share the same characteristics as the columns ``escons``. It seems the election types have similar missing values for `padro`. In fact, we saw that, on the filtered dataset most of the rows have information, therefore we don't need to keep studying the missing values for `padro` as they are all related to the territorial level.

### Missing values in `mesa`

We will study the missing values in `mesa`. We start by seeing if the same territorial levels as `escons` have the same percentage of missing values:

In [44]:
missing_column = "mesa"
(
    100
    * df[df[missing_column].isnull()].value_counts(
        "id_nivell_territorial"
    )
    / df["id_nivell_territorial"].value_counts()
).sort_values(ascending=False)

id_nivell_territorial
CL        100.0
SE    99.998242
DM    99.992698
MU    99.991006
CO    99.847561
CA    97.916667
PR    97.916667
VE    96.969697
ME         <NA>
Name: count, dtype: Float64

We can see that for electoral table (`ME`) we don't have any missing value. But for the other territorial levels, the percentage of missing values is near 100.0%. Therefore, we can say that this column only has values for electoral tables.

### Missing values in `vots_primer_avan` and `vots_segon_avan`

We keep studying the missing values, now it's turn for `vots_primer_avan` and `vots_segon_avan`. First, we check if ``vots_primer_avan`` and ``vots_segon_avan`` have the same missing rows:

In [59]:
print(
    df[
        df["vots_primer_avan"].isnull()
        & df["vots_segon_avan"].notnull()
    ].shape
)
print(
    df[
        df["vots_primer_avan"].notnull()
        & df["vots_segon_avan"].isnull()
    ].shape
)

(0, 27)
(0, 27)


Then we check if the same territorial levels as `escons` have the same percentage of missing values:

In [53]:
missing_column = "vots_primer_avan"
(
    100
    * df[df[missing_column].isnull()].value_counts(
        "id_nivell_territorial"
    )
    / df["id_nivell_territorial"].value_counts()
).sort_values(ascending=False)

id_nivell_territorial
CL    97.729516
CO    26.422764
MU    25.568322
SE    23.434519
DM    23.240234
ME    19.253415
CA    16.666667
PR    16.666667
VE     6.060606
Name: count, dtype: Float64

We can see that the percentage of missing values on territorial levels `CL`, `DM`, `ME` and `SE` is not the same as in `escons`. In general terms, the percentage of missing values is low, but for the territorial level `CL` the missing values are near 100.0%.

For the next analysis, we will use the original dataset.

In [57]:
missing_column = "vots_primer_avan"
print(
    (
        100
        * df[df[missing_column].isnull()].value_counts("type")
        / df["type"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df[df[missing_column].isnull()].value_counts("territori_nom")
        / df["territori_nom"].value_counts()
    )
    .sort_values(ascending=False)
    .dropna()
)
print(
    (
        100
        * df[df[missing_column].isnull()].value_counts("year")
        / df["year"].value_counts()
    )
    .dropna()
)

type
A    36.819079
G    18.514144
M     16.96704
E    12.213045
Name: count, dtype: Float64
territori_nom
Peratallada                    100.0
Vulpellac                      100.0
Fonteta                        100.0
Rupit                          100.0
Pruit                          100.0
                              ...   
Salou                       5.888224
Sant Julià de Cerdanyola    5.511811
Badia del Vallès            4.344049
Palma de Cervelló, la         3.9801
Canonja, la                    3.125
Name: count, Length: 996, dtype: Float64
year
1977    100.000000
1979     99.964912
1980    100.000000
1982     99.961234
1983    100.000000
1984    100.000000
1986     44.862277
1987     99.965523
1988    100.000000
1989      0.117185
1992    100.000000
1993      0.114718
1994      0.117048
1995      0.057989
1996      0.116089
1999      0.076034
2000      0.112546
2003     50.103520
2004      0.108619
2006      0.106885
2008      0.104867
2009      0.109748
2010      0.111133
201

It seems that these variables weren't collected for the first elections, but after 1992 this information was collected.

### Missing values in `districte`

Now it's turn for `districte`. First, we check it this column has the same missing rows as the `mesa` column:

In [60]:
print(
    df[
        df["districte"].isnull()
        & df["mesa"].notnull()
    ].shape
)
print(
    df[
        df["districte"].notnull()
        & df["mesa"].isnull()
    ].shape
)

(0, 27)
(282282, 27)


They are not completely equivalent, but they share a lot of missing rows. When ``districte`` is missing, ``mesa`` is also missing. But when ``mesa`` is missing, ``districte`` is not missing.

We continue the analysis by using the original dataset:

In [61]:
missing_column = "districte"
print(
    (
        100
        * df[df[missing_column].isnull()].value_counts(
            "id_nivell_territorial"
        )
        / df["id_nivell_territorial"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df[df[missing_column].isnull()].value_counts("type")
        / df["type"].value_counts()
    ).sort_values(ascending=False)
)
print(
    (
        100
        * df[df[missing_column].isnull()].value_counts("territori_nom")
        / df["territori_nom"].value_counts()
    )
    .sort_values(ascending=False)
    .dropna()
)

id_nivell_territorial
CL        100.0
MU    99.973017
CO    99.847561
CA    97.916667
PR    97.916667
VE    96.969697
DM         <NA>
ME         <NA>
SE         <NA>
Name: count, dtype: Float64
type
G      9.3828
A    7.997792
M    7.427505
E    6.661205
Name: count, dtype: Float64
territori_nom
Pla d'Urgell                      100.0
Anoia                             100.0
Pallars Jussà                     100.0
Vallès Oriental                   100.0
Maresme                           100.0
                                 ...   
Santa Coloma de Gramenet       1.014007
Reus                           0.997217
Badalona                       0.993341
Hospitalet de Llobregat, l'    0.847199
Barcelona                      0.611357
Name: count, Length: 1005, dtype: Float64


We always have information for the territorial levels municipal district (`DM`), electoral table (`ME`) and section (`SE`). But, for the other territorial levels, the percentage of missing values is near 100.0%.

### Missing values in `seccio`

Finally, we will study the missing values in `seccio`. First, we check it this column has the same missing rows as the `escons` column:

In [62]:
missing_column = "seccio"
(
    100
    * df[df[missing_column].isnull()].value_counts(
        "id_nivell_territorial"
    )
    / df["id_nivell_territorial"].value_counts()
).sort_values(ascending=False)

id_nivell_territorial
CL        100.0
VE    96.969697
DM    38.574297
CO    34.349593
MU    34.155555
CA    33.333333
PR    33.333333
ME         <NA>
SE         <NA>
Name: count, dtype: Float64

We always have information for the territorial levels electoral table (`ME`) and section (`SE`). But, for the other territorial levels, the percentage of missing values is near 100.0%.